In [1]:
import matplotlib
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
current_dir = os.path.dirname(os.path.abspath('../'))
if not current_dir in sys.path:
    sys.path.append(current_dir)

from utils.structures import Pipeline, Deploy
from utils.data_management import dict2str
from utils.machine_learning import one_hot_encoder, one_hot_decoder
from typing import *
import tensorflow as tf
from sklearn.datasets import make_classification
import mne
from combiners import EpochsCombiner
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from utils.machine_learning.designer import ModelDesign, ParallelDesign, LayerDesign
from utils.machine_learning.analyzer import ModelAnalyzer, LFCNNAnalyzer
from mne.datasets import multimodal
import sklearn
import mneflow as mf
import tensorflow as tf
from mneflow.layers import DeMixing, LFTConv, TempPooling, Dense
from mneflow.models import BaseModel
import mneflow
import logging
from time import perf_counter

logger= logging.getLogger(__name__)
logging.root.handlers = []
logger.setLevel(logging.NOTSET)
logging.basicConfig(
    format='%(asctime)s, %(name)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S',
    level=logging.DEBUG,
    handlers=[
        logging.FileHandler('./history.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

# %matplotlib qt

2022-08-27 23:13:29.387037: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-27 23:13:29.387064: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [24]:
class Deconw(tf.keras.layers.Layer):
    def __init__(
        self,
        units=32,
        kernel_size=(4, 10),
        strides=(1, 1),
        padding='valid',
        output_padding=None,
        data_format=None,
        dilation_rate=(1, 1),
        activation=None,
        use_bias=True,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        kernel_regularizer=None,
        bias_regularizer=None,
        activity_regularizer=None,
        kernel_constraint=None,
        bias_constraint=None,
        **kwargs
    ):
        super().__init__()
        self.units = units
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        self.output_padding = output_padding
        self.data_format = data_format
        self.dilation_rate = dilation_rate
        self.activation = activation
        self.use_bias = use_bias
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.kernel_regularizer = kernel_regularizer
        self.bias_regularizer = bias_regularizer
        self.activity_regularizer = activity_regularizer
        self.kernel_constraint = kernel_constraint
        self.bias_constraint = bias_constraint
        self.kwargs = kwargs

    def build(self, input_shape):
        self.n_channels = input_shape[-1]
        self.deconws = self.deconv_constructor(self.n_channels)

    def __call__(self, inputs):
        self.build(inputs.shape)
        outputs = []
        for i in range(self.n_channels):
            input_ = tf.expand_dims(inputs[:, :, :, i], axis=3)
            outputs.append(self.deconws[i](input_))
        return tf.transpose(tf.stack(outputs), (1, 0, 2, 3, 4))

    def deconv_constructor(self, n_channels):
        return [
            tf.keras.layers.Conv2DTranspose(
                filters=1,
                kernel_size=self.kernel_size,
                strides=self.strides,
                padding=self.padding,
                output_padding=self.output_padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate,
                activation=self.activation,
                use_bias=self.use_bias,
                kernel_initializer=self.kernel_initializer,
                bias_initializer=self.bias_initializer,
                kernel_regularizer=self.kernel_regularizer,
                bias_regularizer=self.bias_regularizer,
                activity_regularizer=self.activity_regularizer,
                kernel_constraint=self.kernel_constraint,
                bias_constraint=self.bias_constraint,
                **self.kwargs
            )
        for _ in range(n_channels)
    ]

In [3]:
mne.set_log_level(verbose='CRITICAL')
fname_raw = os.path.join(multimodal.data_path(), 'multimodal_raw.fif')
raw = mne.io.read_raw_fif(fname_raw)
cond = raw.acqparser.get_condition(raw, None)
condition_names = [k for c in cond for k,v in c['event_id'].items()]
epochs_list = [mne.Epochs(raw, **c) for c in cond]
epochs = mne.concatenate_epochs(epochs_list)
epochs = epochs.pick_types(meg='grad')
X = np.array([])
Y = list()
for i, epochs in enumerate(epochs_list):
    data = epochs.get_data()
    if i == 0:
        X = data.copy()
    else:
        X = np.append(X, data, axis=0)
    Y += [i for _ in range(data.shape[0])]

Y = np.array(Y)
X = np.array([X[i, epochs._channel_type_idx['grad'], :] for i, _ in enumerate(X)])
original_X = X.copy()
original_Y = Y.copy()

In [4]:
import_opt = dict(savepath='../tfr/',
    out_name='mne_sample_epochs',
    fs=600,
    input_type='trials',
    target_type='int',
    picks={'meg':'grad'},
    scale=True,  # apply baseline_scaling
    crop_baseline=True,  # remove baseline interval after scaling
    decimate=None,
    scale_interval=(0, 60),  # indices in time axis corresponding to baseline interval
    n_folds=5,
    overwrite=True,
    segment=False,
)

specs = dict()
specs.setdefault('filter_length', 7)
specs.setdefault('n_latent', 4)
specs.setdefault('pooling', 10)
specs.setdefault('stride', 2)
specs.setdefault('padding', 'SAME')
specs.setdefault('pool_type', 'max')
specs.setdefault('nonlin', tf.nn.relu)
specs.setdefault('l1', 3e-4)
specs.setdefault('l2', 0)
specs.setdefault('l1_scope', ['fc', 'dmx', 'tconv', 'fc'])
specs.setdefault('l2_scope', [])
specs.setdefault('maxnorm_scope', [])
specs.setdefault('dropout', .5)

specs['filter_length'] = 17
specs['pooling'] = 5
specs['stride'] = 5
specs['l1'] = 3e-3
out_dim = len(np.unique(original_Y))
Y = original_Y.copy()
Y = one_hot_encoder(Y)
X = original_X.copy()
X = np.transpose(np.expand_dims(X, axis = 1), (0, 1, 3, 2))
print(X.shape)
n_samples, _, n_times, n_channels = X.shape
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(original_X, original_Y, train_size=.85)

# write TFRecord files and metadata file to disk
meta = mneflow.produce_tfrecords((original_X, original_Y), **import_opt)
dataset = mneflow.Dataset(meta, train_batch=100)

(940, 1, 361, 204)
importing from tuple
input shapes: X- (940, 204, 361) targets- (940, 1)
Preprocessing:
Scaling to interval 0.0 - 60.0
Splitting sets
Preprocessed: (940, 1, 361, 204) (940, 8) folds: 5 x 188
940
Prepocessed sample shape: (1, 361, 204)
Target shape actual/metadata:  (8,) (8,)
Saving TFRecord# 0


2022-08-27 23:14:20.429734: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-27 23:14:20.429769: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-27 23:14:20.429787: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (arcolinux-machine): /proc/driver/nvidia/version does not exist
2022-08-27 23:14:20.430040: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
lfcnnd = ModelDesign(
    None,
    DeMixing(
        size=specs['n_latent'],
        nonlin=tf.identity,
        axis=3, specs=specs
    ),
    LFTConv(
        size=specs['n_latent'],
        nonlin=specs['nonlin'],
        filter_length=specs['filter_length'],
        padding=specs['padding'],
        specs=specs
    ),
    TempPooling(
        pooling=specs['pooling'],
        pool_type=specs['pool_type'],
        stride=specs['stride'],
        padding=specs['padding'],
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)

class ZubarevNet(BaseModel):
    def __init__(self, Dataset, specs=dict(), design=lfcnnd, design_name='design'):
        self.scope = design_name
        self.design = design
        specs.setdefault('filter_length', 7)
        specs.setdefault('n_latent', 4)
        specs.setdefault('pooling', 4)
        specs.setdefault('stride', 4)
        specs.setdefault('padding', 'SAME')
        specs.setdefault('pool_type', 'max')
        specs.setdefault('nonlin', tf.nn.relu)
        specs.setdefault('l1', 3e-4)
        specs.setdefault('l2', 0)
        specs.setdefault('l1_scope', ['fc', 'demix', 'lf_conv'])
        specs.setdefault('l2_scope', [])
        specs.setdefault('maxnorm_scope', [])

        super().__init__(Dataset, specs)

    def build_graph(self):
        return self.design(self.inputs)

    def set_design(self, design: ModelDesign):
        self.design = design

Setting reg for fc, to l1


In [48]:
simplenetd = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    DeMixing(
        size=specs['n_latent'],
        nonlin=tf.identity,
        axis=3, specs=specs
    ),
    LFTConv(
        size=specs['n_latent'],
        nonlin=specs['nonlin'],
        filter_length=specs['filter_length'],
        padding=specs['padding'],
        specs=specs
    ),
    LFTConv(
        size=specs['n_latent'],
        nonlin=specs['nonlin'],
        filter_length=specs['filter_length'],
        padding=specs['padding'],
        specs=specs
    ),
    LayerDesign(
        lambda X: X[:, :, ::2, :]
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)

lfrnnd = ModelDesign(
    None,
    LayerDesign(tf.squeeze, axis=1),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            specs['n_latent'],
            bias_regularizer='l1',
            return_sequences=True,
            kernel_regularizer=tf.keras.regularizers.L1(.01),
            recurrent_regularizer=tf.keras.regularizers.L1(.01),
            dropout=0.4,
            recurrent_dropout=0.4,
        ),
        merge_mode='sum'
    ),
    LayerDesign(tf.expand_dims, axis=1),
    LFTConv(
        size=specs['n_latent'],
        nonlin=specs['nonlin'],
        filter_length=specs['filter_length'],
        padding=specs['padding'],
        specs=specs
    ),
    TempPooling(
        pooling=specs['pooling'],
        pool_type=specs['pool_type'],
        stride=specs['stride'],
        padding=specs['padding'],
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)


# newnetd = ModelDesign(
#     tf.keras.Input(shape=(1, n_times, n_channels)),
#     Deconw(kernel_size=(specs['n_latent'], specs['filter_length']), activation='relu', kernel_regularizer='l1'),
#     tf.keras.layers.Conv2D(1, (1, specs['filter_length']), activation='relu', kernel_regularizer='l2'),
#     LayerDesign(
#         lambda X: tf.transpose(tf.squeeze(X, axis=-1), (0, 2, 3, 1))
#     ),
#     tf.keras.layers.Conv2D(1, (1, 204), padding='same'),
#     LayerDesign(
#         lambda X: tf.transpose(X, (0, 3, 2, 1))
#     ),
#     LayerDesign(
#         lambda X: X[:, :, ::specs['pooling'], :]
#     ),
#     tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
#     Dense(size=out_dim, nonlin=tf.identity, specs=specs)
# )

# newnetd = ModelDesign(
#     tf.keras.Input(shape=(1, n_times, n_channels)),
#     Deconw(kernel_size=(specs['n_latent'], specs['filter_length']), activation='relu', kernel_regularizer='l1'),
#     tf.keras.layers.Conv2D(1, (1, specs['filter_length']), activation='relu'),
#     LayerDesign(
#         lambda X: tf.transpose(tf.squeeze(X, axis=-1), (0, 1, 3, 2))
#     ),
#     tf.keras.layers.DepthwiseConv2D((204, 1), kernel_regularizer='l1'),
#     LayerDesign(
#         lambda X: X[:, :, ::2, :]
#     ),
#     tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
#     Dense(size=out_dim, nonlin=tf.identity, specs=specs)
# )

newnetd = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    Deconw(kernel_size=(specs['n_latent'], specs['filter_length']), activation='relu', kernel_regularizer='l1'),
    LayerDesign(
        lambda X: tf.transpose(tf.squeeze(X, axis=-1), (0, 1, 3, 2))
    ),
    tf.keras.layers.DepthwiseConv2D((1, specs['filter_length']), activation='relu', depthwise_regularizer='l1'),
    tf.keras.layers.DepthwiseConv2D((n_channels, 1), name='demixing'),
    LayerDesign(
        lambda X: X[:, :, ::2, :]
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)

Setting reg for fc, to l1
Setting reg for fc, to l1
Setting reg for fc, to l1


In [49]:
# model = ZubarevNet(dataset, specs, lfcnnd, 'lfcnn')
model = ZubarevNet(dataset, specs, newnetd, 'new')
model.build()
t1 = perf_counter()
model.train(n_epochs=25, eval_step=100, early_stopping=5)
runtime = perf_counter() - t1

Built: fc input: (None, 1, 181, 4)
Input shape: (1, 361, 204)
y_pred: (None, 8)
Initialization complete!
Epoch 1/25
100/100 - 250s - loss: 15.5052 - cat_ACC: 0.1535 - val_loss: 13.6132 - val_cat_ACC: 0.1223 - 250s/epoch - 3s/step
Epoch 2/25


In [23]:
y_true_train, y_pred_train = model.predict(meta['train_paths'])
y_true_test, y_pred_test = model.predict(meta['test_paths'])
runtime=103.57*60
logging.info(
    f'{model.scope} performance:\n'
    f'\truntime: {runtime : .4f}\n'
    f'\ttrain-set: {sklearn.metrics.accuracy_score(one_hot_decoder(y_true_train), one_hot_decoder(y_pred_train))}\n'
    f'\ttest-set: {sklearn.metrics.accuracy_score(one_hot_decoder(y_true_test), one_hot_decoder(y_pred_test))}'
)

2022-08-28 10:15:07.321252: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 276901440 exceeds 10% of free system memory.
2022-08-28 10:15:07.670907: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 276901440 exceeds 10% of free system memory.


No dataset specified using validation dataset (Default)
10:15:24, root INFO new performance:
	runtime:  6214.2000
	train-set: 0.8680851063829788
	test-set: 0.7712765957446809


In [42]:
n_samples, _, n_times, n_channels = (940, 1, 361, 204)
out_dim=8

# newnetd = ModelDesign(
#     tf.keras.Input(shape=(1, n_times, n_channels)),
#     Deconw(kernel_size=(specs['n_latent'], specs['filter_length']), activation='relu', kernel_regularizer='l1'),
#     tf.keras.layers.Conv2D(1, (1, specs['filter_length']), activation='relu', kernel_regularizer='l2'),
#     LayerDesign(
#         lambda X: tf.transpose(tf.squeeze(X, axis=-1), (0, 1, 3, 2))
#     ),
#     # TensorShape([None, 4, 361, 204])
#     tf.keras.layers.Conv2D(1, (204, 1), padding='valid', name='demixing'),
#     # LayerDesign(
#     #     lambda X: tf.transpose(X, (0, 3, 2, 1))
#     # ),
#     # LayerDesign(
#     #     lambda X: X[:, :, ::specs['pooling'], :]
#     # ),
#     # tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
#     # Dense(size=out_dim, nonlin=tf.identity, specs=specs)
# )

# newnetd = ModelDesign(
#     tf.keras.Input(shape=(1, n_times, n_channels)),
#     Deconw(kernel_size=(4, 10)),
#     tf.keras.layers.Conv2D(1, (1, 10)),
#     LayerDesign(
#         lambda X: tf.transpose(tf.squeeze(X, axis=-1), (0, 1, 3, 2))
#     ),
#     tf.keras.layers.DepthwiseConv2D((204, 1), name='demixing'),
#     LayerDesign(
#         lambda X: X[:, :, ::2, :]
#     ),
#     tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
#     Dense(size=out_dim, nonlin=tf.identity, specs=specs)
# )


newnetd = ModelDesign(
    tf.keras.Input(shape=(1, n_times, n_channels)),
    Deconw(kernel_size=(4, 10)),
    LayerDesign(
        lambda X: tf.transpose(tf.squeeze(X, axis=-1), (0, 1, 3, 2))
    ),
    tf.keras.layers.DepthwiseConv2D((1, 10)),
    tf.keras.layers.DepthwiseConv2D((204, 1), name='demixing'),
    LayerDesign(
        lambda X: X[:, :, ::2, :]
    ),
    tf.keras.layers.Dropout(specs['dropout'], noise_shape=None),
    Dense(size=out_dim, nonlin=tf.identity, specs=specs)
)


print('input_shape: ', (1, n_times, n_channels))
newnetd().shape

Setting reg for fc, to l1
input_shape:  (1, 361, 204)
Built: fc input: (None, 1, 181, 4)


TensorShape([None, 8])

In [43]:
inp = tf.keras.Input(shape=(1, n_times, n_channels))
model = tf.keras.Model(inp, newnetd(inp))

In [44]:
dmx = model.get_layer('demixing')

In [45]:
dmx.weights

[<tf.Variable 'demixing/depthwise_kernel:0' shape=(204, 1, 4, 1) dtype=float32, numpy=
 array([[[[ 0.031633  ],
          [ 0.04099584],
          [-0.00537659],
          [ 0.04578142]]],
 
 
        [[[-0.02466786],
          [-0.05742272],
          [-0.03772743],
          [ 0.00679388]]],
 
 
        [[[ 0.02253095],
          [-0.02841793],
          [-0.03629241],
          [-0.00137744]]],
 
 
        [[[-0.02788217],
          [ 0.01266242],
          [-0.00487591],
          [-0.03959247]]],
 
 
        [[[-0.0184562 ],
          [ 0.06493163],
          [ 0.00136462],
          [-0.03449699]]],
 
 
        [[[ 0.02535935],
          [-0.0728882 ],
          [-0.05114057],
          [ 0.00465818]]],
 
 
        [[[-0.04673408],
          [-0.03232426],
          [-0.02720003],
          [-0.00286223]]],
 
 
        [[[ 0.02021289],
          [-0.02026704],
          [-0.02710452],
          [-0.01228148]]],
 
 
        [[[-0.06937988],
          [ 0.06833906],
          [-0.0